In [24]:
import numpy as np
import pandas as pd
import MetaTrader5 as mt5

import statsmodels.api as sm
from pandas.tseries.offsets import BDay

import csv
import time
import math
from datetime import datetime, timedelta

import requests
import json

import threading
from threading import Lock

from ta.trend import ADXIndicator

In [2]:
def s_print(*a, **b):
    """Thread safe print function"""
    with s_print_lock:
        print(*a, **b)

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cycler

colors = cycler('color', ['669FEE', '66EE91', '9988DD', 'EECC55', '88BB44', 'FFBBBB'])
plt.rc('figure', facecolor='313233')
plt.rc('axes',  facecolor='313233', edgecolor='none', axisbelow=True, grid=True, prop_cycle=colors, labelcolor='gray')
plt.rc('grid', color='474A4A', linestyle='solid')
plt.rc('xtick', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('legend', facecolor='313233', edgecolor='313233')
plt.rc('text')

In [4]:
# Twitter notification keys

chat_id = "6208180231"
api_key = "6274819941:AAF8U6FnGv5A0DcSBNXCpnPCJPciwSww8-A"

In [5]:
def send_telegram_message(message: str, chat_id: str, api_key: str,):
    responses = {}

    proxies = None
    headers = {'Content-Type': 'application/json',
                'Proxy-Authorization': 'Basic base64'}
    data_dict = {'chat_id': chat_id,
                 'text': message,
                 'parse_mode': 'HTML',
                 'disable_notification': True}
    data = json.dumps(data_dict)
    url = f'https://api.telegram.org/bot{api_key}/sendMessage'
    
    requests.packages.urllib3.disable_warnings()
    response = requests.post(url,
                             data=data,
                             headers=headers,
                             proxies=proxies,
                             verify=False)
    return response

In [6]:
def get_bars(symbol, start_date, end_date, timeframe=mt5.TIMEFRAME_M1):
    mt5.initialize()
    bars = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    df_bars = pd.DataFrame(bars)
    df_bars["time"] = pd.to_datetime(df_bars["time"], unit="s")
    return df_bars

In [7]:
def get_latest_bar(symbol, timeframe=mt5.TIMEFRAME_M1):
    mt5.initialize()
    bar = mt5.copy_rates_from(
        symbol, 
        timeframe, 
        datetime.now() + timedelta(hours = 3), 
        1)  # number of bars
    df_bar = pd.DataFrame(bar)
    df_bar["time"] = pd.to_datetime(df_bar["time"], unit="s")
    return df_bar

In [8]:
def get_prices(location):
    mt5.initialize()

    symbol_list = []
    for symbol in mt5.symbols_get():
        if location in symbol.path:
            symbol_list.append(symbol)
            
    return symbol_list

In [9]:
def new_bar_event(symbol, last_bar):
    mt5.initialize()
    
    # Fixing a formatting error when getting last_bar['time'].iloc[0]
    date_str = last_bar['time'].iloc[0]
    date_format = '%Y-%m-%d %H:%M:%S'

    last_bar_time_formatted = datetime.strptime(date_str, date_format)
    while True:
        new_bar = get_latest_bar(symbol)        
        if new_bar['time'].iloc[0] != last_bar_time_formatted:
            return new_bar
        
        time.sleep(10)

In [10]:
def get_num_of_lots(name, leverage=30, risk=0.01):
    ask_per_fraction = (mt5.symbols_get(name)[0].ask * mt5.symbols_get(name)[0].trade_contract_size) * mt5.symbols_get(name)[0].volume_min
    ask_per_fraction_m = ask_per_fraction / leverage
    capital_at_risk = mt5.account_info().balance * risk
    
    order_size = math.floor(capital_at_risk / ask_per_fraction_m) * mt5.symbols_get(symbol)[0].volume_min
    return  order_size

In [11]:
def find_filling_mode(symbol):
    mt5.initialize()
    
    for i in range(2):
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC}
 
        result = mt5.order_check(request)
            
        if result.comment == "Done":
            break
 
    return i

In [12]:
def open_long(name, lot_size):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_BUY,   # mt5.ORDER_TYPE_SELL
            "price" : mt5.symbol_info_tick(name).ask, # mt5.symbol_info_tick(symbol).bid 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
    
    result = mt5.order_send(request)
    
    time.sleep(2)
    return result

In [13]:
def close_long(name, lot_size, order_id):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "position" : order_id,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_SELL,
            "price" : mt5.symbol_info_tick(name).bid, 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
    
    result = mt5.order_send(request)
    
    time.sleep(2)
    return result

In [14]:
def open_short(name, lot_size):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_SELL,
            "price" : mt5.symbol_info_tick(name).bid, 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
    
    result = mt5.order_send(request)
    
    time.sleep(2)
    return result

In [15]:
def close_short(name, lot_size, order_id):
    mt5.initialize()
    filling_type = find_filling_mode(name)
    request = {
            "action" : mt5.TRADE_ACTION_DEAL,
            "symbol" : name,
            "position" : order_id,
            "volume" : lot_size,
            "type" : mt5.ORDER_TYPE_BUY,   # mt5.ORDER_TYPE_SELL
            "price" : mt5.symbol_info_tick(name).ask, # mt5.symbol_info_tick(symbol).bid 
            "deviation" : 10,
            "type_filling" : filling_type,
            "type_time " : mt5.ORDER_TIME_GTC
        }
        
    result = mt5.order_send(request)   # Sends order to broker
    
    time.sleep(2)
    return result  

In [16]:
def get_ticket(name):
    for position in mt5.positions_get():
        if position.symbol == name:
            return position.ticket
        
    return

In [17]:
def get_price_open(name):
    for position in mt5.positions_get():
        if position.symbol == name:
            return position.price_open
        
    return

In [19]:
def trade_engine(name, lot_size, data, position_type):
    mt5.initialize()
    
    # Get close
    data_close = data['close']
    
    filtered_data_close_cycle, filtered_data_close_trend = sm.tsa.filters.hpfilter(data_close)
        
    # Construct slow MA, slow STD, and fast MA
    window_slow = 50
    window_fast = 10
    slow_ma = filtered_data_close_trend.ewm(span=window_slow).mean()
    slow_std = filtered_data_close_trend.ewm(span=window_slow).std()
    fast_ma = filtered_data_close_trend.ewm(span=window_fast).mean()
    
    
    # Contruct trade parameters
    upper_threshold = slow_ma + (slow_std * 0.3)
    lower_threshold = slow_ma - (slow_std * 0.3)
        
          
    # Position type encoding
    
    LONG = 1
    FREE = 0
    SHORT = -1
      
    # If Long
    if position_type == LONG:
            
        if (fast_ma.iloc[-1] <= slow_ma.iloc[-1]):
            # CLOSE LONG POSITION AT A GAIN
            ticket = get_ticket(name)
            result = close_long(name, lot_size, ticket)
            send_telegram_message(f'{name}: Closed Long Position: {data_close.iloc[-1]}, Account Balance: {mt5.account_info().balance}.', chat_id, api_key)
            position_type = FREE
                                
    elif position_type == SHORT:        
        if (fast_ma.iloc[-1] >= slow_ma.iloc[-1]):
            # CLOSE SHORT POSITION AT A GAIN
            ticket = get_ticket(name)
            result = close_short(name, lot_size, ticket)
            send_telegram_message(f'{name}: Closed Short Position: {data_close.iloc[-1]}, Account Balance: {mt5.account_info().balance}.', chat_id, api_key)
            position_type = FREE
                
                
    elif position_type == FREE:
        # cross above -> long

        if (fast_ma.iloc[-1] >= upper_threshold.iloc[-1]):
            # OPEN LONG POSITION
            result = open_long(name, lot_size)
            send_telegram_message(f'{name}: Openned Long Position: {data_close.iloc[-1]}, Account Balance: {mt5.account_info().balance}.', chat_id, api_key)
            position_type = LONG
                
            # cross below -> short
        elif (fast_ma.iloc[-1] <= lower_threshold.iloc[-1]):
            # OPEN SHORT POSITION
            result = open_short(name, lot_size)
            send_telegram_message(f'{name}: Openned Short Position: {data_close.iloc[-1]}, Account Balance: {mt5.account_info().balance}.', chat_id, api_key)
            position_type = SHORT
                
    return position_type
    

In [25]:
def momentum_trader(params):
    
    try:
        symbol = params[0]
        lot_size = params[1]

        ##################################################
        # Set Parameters

        end_date = datetime.now() + timedelta(hours = 3)
        start_date = end_date - BDay(1)

        mt5.initialize()

        symbol_csv = symbol + '.csv'

        ##################################################
        # Generate a days worth of market data and put it into a csv

        symbol_object = mt5.symbols_get(symbol)
        symbol_name = symbol_object[0].name

        symbol_data = get_bars(
            symbol_name, 
            start_date, 
            end_date)

        symbol_data.to_csv(symbol_csv, index=False)

        data = pd.read_csv(symbol_csv)

        ##################################################
        # Trade Loop

        position_type = 0
        while True:
            last_bar = data.tail(1)
            new_bar = new_bar_event(symbol_name, last_bar)

            new_bar.to_csv(symbol_csv, mode='a', index=False, header=False)
            data = pd.read_csv(symbol_csv)


            position_type = trade_engine(symbol_name, lot_size, data, position_type)

        ##################################################
        
    except Exception as e:
        logging.error(traceback.format_exc())

In [21]:
s_print_lock = Lock()

portfolio = [
    ('EURUSD', 0.16),
    ('US30', 0.4),
    ('ETHUSD', 0.6),
    ('Sugar', 900)
]


# Loop through the list and create a new thread for each item

mt5.initialize()
print('################################################################')
print(f'\t\tBEGINNING BALANCE TODAY: {mt5.account_info().balance}')
print('################################################################')
print()

for params in portfolio:
    t = threading.Thread(target=momentum_trader, args=(params,))
    t.start()
    
    


################################################################
		BEGINNING BALANCE TODAY: 19854.76
################################################################

